In [ ]:
## Creating a YAML file from YAML files in a folder

import os
import re
import yaml

def parse_md_file(md_path):
    """
    Parse a markdown file to extract epic name and main tasks (e.g., 1.0, 2.0).
    Returns:
        epic_name (str), list of main task titles
    """
    epic_name = None
    main_tasks = []

    with open(md_path, encoding="utf-8") as file:
        for line in file:
            # Extract epic/module name
            if line.strip().startswith("> **Module:**"):
                match = re.search(r'> \*\*Module:\*\*\s*(.+)', line)
                if match:
                    epic_name = match.group(1).strip()

            # Extract top-level task (X.0 only)
            match = re.match(r'- \[ \] (\d+\.0)\s+(.*)', line.strip())
            if match:
                task_title = match.group(2).strip()
                main_tasks.append(task_title)

    return epic_name, main_tasks


def parse_folder_to_yaml(folder_path):
    """
    Parse all .md files in a folder and return a YAML string
    with epic and main task mappings per file.
    """
    output_data = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".md"):
            full_path = os.path.join(folder_path, filename)
            epic_name, main_tasks = parse_md_file(full_path)

            if epic_name and main_tasks:
                output_data.append({
                    "file": filename,
                    "epic": epic_name,
                    "main_tasks": main_tasks
                })

    return yaml.dump(output_data, sort_keys=False, allow_unicode=True)


if __name__ == "__main__":
    folder_path = "./implementation"  # 🔁 Replace with your folder path
    output_yaml = parse_folder_to_yaml(folder_path)

    with open("epics_and_tasks.yaml", "w", encoding="utf-8") as out_file:
        out_file.write(output_yaml)
a
    print("✅ YAML file generated: epics_and_tasks.yaml")


✅ YAML file generated: epics_and_tasks.yaml


In [ ]:
# parsing ymal file to generate epic and tasks and subtasks

import os
import re
import yaml


def parse_md_file(md_path):
    """
    Parse a markdown file to extract epic name, stories (main tasks),
    and subtasks under each story.

    Returns:
        epic_name (str), list of dicts with story and their subtasks
    """
    epic_name = None
    stories = []
    current_story = None

    with open(md_path, encoding="utf-8") as file:
        for line in file:
            line = line.strip()

            # Extract epic/module name
            if line.startswith("> **Module:**"):
                match = re.search(r"> \*\*Module:\*\*\s*(.+)", line)
                if match:
                    epic_name = match.group(1).strip()

            # Match main story (e.g., 1.0, 2.0)
            story_match = re.match(r"- \[ \] (\d+\.0)\s+(.*)", line)
            if story_match:
                current_story = {"story": story_match.group(2).strip(), "tasks": []}
                stories.append(current_story)
                continue

            # Match subtask (e.g., 1.1, 2.2)
            subtask_match = re.match(r"- \[ \] (\d+\.\d+)\s+(.*)", line)
            if (
                subtask_match
                and current_story
                and not subtask_match.group(1).endswith(".0")
            ):
                subtask_title = subtask_match.group(2).strip()
                current_story["tasks"].append(subtask_title)

    return epic_name, stories


def parse_folder_to_yaml(folder_path):
    """
    Parse all .md files in a folder and return a YAML string
    with epics, stories, and subtasks.
    """
    output_data = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".md"):
            full_path = os.path.join(folder_path, filename)
            epic_name, stories = parse_md_file(full_path)

            if epic_name and stories:
                output_data.append({
                    "file": filename,
                    "epic": epic_name,
                    "stories": stories,
                })

    return yaml.dump(output_data, sort_keys=False, allow_unicode=True)


if __name__ == "__main__":
    folder_path = "./implementation"  # Replace with your folder path
    output_yaml = parse_folder_to_yaml(folder_path)

    with open("epics_stories_subtasks.yaml", "w", encoding="utf-8") as out_file:
        out_file.write(output_yaml)

    print("✅ YAML file generated: epics_stories_subtasks.yaml")


✅ YAML file generated: epics_stories_subtasks.yaml


In [ ]:
import os
import yaml
from dotenv import load_dotenv
from jira import JIRA

# Load API token from .env
load_dotenv()
jira_token = "ATATT3xFfGF0CeDTD6VZufZil2T7dchVORcpdxuaktUQSN7cvoIRhmPjBiu9i6m4GktGyhyPlf7uC_yM1oPB9SuiRaF-OXA8BQIJ_irybwEnlocr2EbYkoYlyzcg59BGeAFStRKMapY0nefLW66AkKH2HyptWRpNtpN2GpylQLaanMGe_cM6Gc8=933BDEA8"  # 🔐 Replace with your real token

# Jira setup
jira_server = "https://divami.atlassian.net"
jira_email = "yeshwanth@divami.com"
project_key = "AJT"


auth = HTTPBasicAuth(jira_email, jira_token)
headers = {"Accept": "application/json", "Content-Type": "application/json"}

url = f"{jira_server}/rest/api/2/issue"


def get_epic_link_field():
    """Get the Epic Link field ID for your JIRA instance"""
    field_url = f"{jira_server}/rest/api/2/field"
    try:
        response = requests.get(field_url, headers=headers, auth=auth)
        response.raise_for_status()
        fields = response.json()

        # Look for Epic Link field
        for field in fields:
            if (
                "epic" in field.get("name", "").lower()
                and "link" in field.get("name", "").lower()
            ):
                print(f"Found Epic Link field: {field['name']} - ID: {field['id']}")
                return field["id"]

        # Common Epic Link field IDs if not found by name
        common_epic_fields = [
            "customfield_10014",
            "customfield_10008",
            "customfield_10011",
            "customfield_10009",
        ]
        print(
            "Epic Link field not found by name. Trying common IDs:", common_epic_fields
        )
        return "customfield_10014"  # Most common default

    except requests.exceptions.RequestException as e:
        print(f"❌ Error getting fields: {e}")
        return "customfield_10014"  # Default fallback


def load_yaml(file_path):
    """Load and parse YAML file"""
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            return yaml.safe_load(file)
    except FileNotFoundError:
        print(f"❌ File {file_path} not found.")
        return None
    except yaml.YAMLError as e:
        print(f"❌ Error parsing YAML: {e}")
        return None


def gen_epic(epic_name):
    """Generate Epic in JIRA"""
    issue_data = {
        "fields": {
            "project": {"key": "D4"},
            "summary": epic_name,
            "description": "EPIC: Auto-generated from Python.",
            "issuetype": {"name": "Epic"},
            # "customfield_10011": epic_name  # Uncomment if you need Epic Name field
        }
    }

    try:
        response = requests.post(
            url, headers=headers, auth=auth, data=json.dumps(issue_data)
        )
        response.raise_for_status()
        issue_key = response.json().get("key")
        print(f"✅ Created Epic: {issue_key}")
        return issue_key
    except requests.exceptions.RequestException as e:
        print(f"❌ Error creating epic: {e}")
        if hasattr(e, "response") and e.response is not None:
            print(f"Response content: {e.response.text}")
        return None


def gen_story(story_title, epic_key=None):
    """Generate Story in JIRA"""
    issue_data = {
        "fields": {
            "project": {"key": "D4"},
            "summary": story_title,
            "description": "STORY: Auto-generated from Python.",
            "issuetype": {"name": "Story"},
        }
    }

    # Link story to Epic using Epic Link field
    if epic_key:
        epic_link_field = get_epic_link_field()
        issue_data["fields"][epic_link_field] = epic_key

    try:
        response = requests.post(
            url, headers=headers, auth=auth, data=json.dumps(issue_data)
        )
        response.raise_for_status()
        issue_key = response.json().get("key")
        print(f"✅ Created Story: {issue_key}")
        return issue_key
    except requests.exceptions.RequestException as e:
        print(f"❌ Error creating story: {e}")
        if hasattr(e, "response") and e.response is not None:
            print(f"Response content: {e.response.text}")
        return None


def gen_task(task_name, story_key=None):
    """Generate Task in JIRA as a Subtask"""
    issue_data = {
        "fields": {
            "project": {"key": "D4"},
            "summary": task_name,
            "description": "TASK: Auto-generated from Python.",
            "issuetype": {"name": "Sub-task"},  # Changed to Sub-task
        }
    }

    # Link task to Story as parent (for subtasks)
    if story_key:
        issue_data["fields"]["parent"] = {"key": story_key}

    try:
        response = requests.post(
            url, headers=headers, auth=auth, data=json.dumps(issue_data)
        )
        response.raise_for_status()
        issue_key = response.json().get("key")
        print(f"✅ Created Task (Sub-task): {issue_key}")
        return issue_key
    except requests.exceptions.RequestException as e:
        print(f"❌ Error creating task: {e}")
        if hasattr(e, "response") and e.response is not None:
            print(f"Response content: {e.response.text}")

        # If Sub-task fails, try creating as regular Task
        print("🔄 Retrying as regular Task...")
        issue_data["fields"]["issuetype"] = {"name": "Task"}

        # For regular tasks, we might need to link differently
        if story_key:
            # Remove parent field for regular tasks
            issue_data["fields"].pop("parent", None)
            # Try linking as Epic Link instead
            epic_link_field = get_epic_link_field()
            issue_data["fields"][epic_link_field] = story_key

        try:
            response = requests.post(
                url, headers=headers, auth=auth, data=json.dumps(issue_data)
            )
            response.raise_for_status()
            issue_key = response.json().get("key")
            print(f"✅ Created Task: {issue_key}")
            return issue_key
        except requests.exceptions.RequestException as e2:
            print(f"❌ Error creating task (retry): {e2}")
            if hasattr(e2, "response") and e2.response is not None:
                print(f"Response content: {e2.response.text}")
            return None


def parse_yaml(yaml_data):
    """Parse YAML data and create JIRA issues"""
    if not yaml_data:
        print("❌ YAML is empty.")
        return

    for entry in yaml_data:
        epic_name = entry.get("epic")
        stories = entry.get("stories", [])

        print(f"\n🔹 Epic: {epic_name}")
        epic_key = gen_epic(epic_name)

        if not epic_key:
            print("❌ Skipping due to epic creation failure.")
            continue

        for i, story in enumerate(stories, 1):
            story_title = story.get("story")  # Changed from "title" to "story"
            tasks = story.get("tasks", [])  # Changed from "subtasks" to "tasks"

            print(f"  {i}. Story: {story_title}")
            story_key = gen_story(story_title, epic_key)

            if not story_key:
                print(f"  ⚠️ Skipping tasks (story creation failed).")
                continue

            for j, task in enumerate(tasks, 1):
                print(f"    {i}.{j} Task: {task}")
                gen_task(task, story_key)


def main():
    """Main function"""
    print("🔍 Checking JIRA field configuration...")
    epic_link_field = get_epic_link_field()
    print(f"Using Epic Link field: {epic_link_field}")

    yaml_data = load_yaml("epics_stories_subtasks.yaml")
    if yaml_data:
        parse_yaml(yaml_data)
        print("\n✅ YAML data processed successfully.")
    else:
        print("❌ Failed to load YAML data.")


if __name__ == "__main__":
    main()

🔍 Checking JIRA field configuration...
Found Epic Link field: Epic Link - ID: customfield_10008
Using Epic Link field: customfield_10008

🔹 Epic: infrastructure-devops
✅ Created Epic: D4-120
  1. Story: Cloud Infrastructure Foundation
Found Epic Link field: Epic Link - ID: customfield_10008
✅ Created Story: D4-121
    1.1 Task: Set up multi-cloud strategy with primary cloud provider configuration
✅ Created Task (Sub-task): D4-122
    1.2 Task: Initialize Terraform infrastructure-as-code repository structure
✅ Created Task (Sub-task): D4-123
    1.3 Task: Configure VPC, subnets, and networking infrastructure
✅ Created Task (Sub-task): D4-124
    1.4 Task: Set up security groups, NACLs, and firewall rules
✅ Created Task (Sub-task): D4-125
    1.5 Task: Configure DNS management and SSL certificate automation
✅ Created Task (Sub-task): D4-126
    1.6 Task: Set up container registry with security scanning integration
✅ Created Task (Sub-task): D4-127
    1.7 Task: Create environment-specifi